In [1]:
import os
import pandas as pd
import glob
import numpy as np

In [2]:
gtf='/global/scratch/users/chandlersutherland/Athaliana/GTFs/Araport11_GTF_genes_transposons.current.gtf'
gtf_import=pd.read_csv(gtf,sep='\t', skiprows=3).reset_index()
gtf_import

,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,##GTF
0,Chr1,Araport11,gene,3631,5899,.,+,.,"transcript_id ""AT1G01010""; gene_id ""AT1G01010"";"
1,Chr1,Araport11,mRNA,3631,5899,.,+,.,"transcript_id ""AT1G01010.1""; gene_id ""AT1G01010"";"
2,Chr1,Araport11,CDS,3760,3913,.,+,0,"transcript_id ""AT1G01010.1""; gene_id ""AT1G01010"";"
3,Chr1,Araport11,CDS,3996,4276,.,+,2,"transcript_id ""AT1G01010.1""; gene_id ""AT1G01010"";"
4,Chr1,Araport11,CDS,4486,4605,.,+,0,"transcript_id ""AT1G01010.1""; gene_id ""AT1G01010"";"
...,...,...,...,...,...,...,...,...,...
927385,ChrM,Araport11,exon,230304,230362,.,+,.,"transcript_id ""ATMG01275""; gene_id ""ATMG01275"";"
927386,ChrM,Araport11,exon,233874,234132,.,+,.,"transcript_id ""ATMG01275""; gene_id ""ATMG01275"";"
927387,ChrM,Araport11,gene,264467,265516,.,-,.,"transcript_id ""ATMG01170-2""; gene_id ""ATMG0117..."
927388,ChrM,Araport11,mRNA,264467,265516,.,-,.,"transcript_id ""ATMG01170""; gene_id ""ATMG01170"";"


In [3]:
gtf_import=gtf_import.rename(columns={"level_0": "chrom", "level_1": "Genome", "level_2" : "type", "level_3" : "chromStart", "level_4":"chromEnd", "level_6":'strand'})

In [29]:
gene_import=gtf_import[gtf_import['type'] == 'gene']
gene_import['name'] = gene_import['##GTF'].str.split(';',expand=True)[1].str.split(' ', expand=True)[2].str.replace('"', '')

In [31]:
bed_ready = gtf_import.drop(['Genome', 'type', 'level_5', 'level_7', '##GTF'], axis=1)

In [36]:
bed_ready

,chrom,chromStart,chromEnd,strand,name
0,Chr1,3631,5899,+,AT1G01010
17,Chr1,6788,9130,-,AT1G01020
138,Chr1,11649,13714,-,AT1G01030
157,Chr1,23121,31227,+,AT1G01040
246,Chr1,28500,28706,+,AT1G01046
...,...,...,...,...,...
927371,ChrM,126289,128223,-,ATMG01390
927374,ChrM,361637,361939,+,ATMG00650-2
927377,ChrM,82935,83321,-,ATMG01275-2
927381,ChrM,230304,234132,+,ATMG01275-3


In [38]:
bed_ready.to_csv("/global/scratch/users/chandlersutherland/Athaliana/Atha_genes.bed", sep='\t', header=False, index=False)

Make a CDS only bed file with chromosomes coded as just numbers to play nice with egglib vcf parser

In [7]:
cds_import=gtf_import[gtf_import['type'] == 'CDS']
cds_import['name'] = cds_import['##GTF'].str.split(';',expand=True)[1].str.split(' ', expand=True)[2].str.replace('"', '')
cds_import['chrom']=cds_import['chrom'].str.replace('Chr', '')

bed_ready2 = cds_import.drop(['Genome', 'type', 'level_5', 'level_7', '##GTF'], axis=1)
bed_ready2

/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,chrom,chromStart,chromEnd,strand,name
2,1,3760,3913,+,AT1G01010
3,1,3996,4276,+,AT1G01010
4,1,4486,4605,+,AT1G01010
5,1,4706,5095,+,AT1G01010
6,1,5174,5326,+,AT1G01010
...,...,...,...,...,...
927376,M,361637,361939,+,ATMG00650
927379,M,82937,83321,-,ATMG01275
927383,M,230304,230362,+,ATMG01275
927384,M,233874,234132,+,ATMG01275


In [8]:
bed_ready2.to_csv("/global/scratch/users/chandlersutherland/Athaliana/Atha_cds.bed", sep='\t', header=False, index=False)